In [41]:
import pandas as pd

In [42]:
df_metadata = pd.read_csv('../attribute_extraction_metadata_template/Initial Attribute Definition for First Release - UPDATED SHEET .csv')

In [43]:
df_metadata = df_metadata[['category', 'attribute_field', 'category_attributevalue', 'description', 'max_multi_select', 'usage', 'entry mode']]

In [44]:
metadata_category = set(df_metadata.category)

In [45]:
category2config = {}
for i in metadata_category:
    df_tmp = df_metadata[df_metadata.category == i]
    category2config[i] = sorted(df_tmp.to_dict('records'), key=lambda x: x['attribute_field'])

In [46]:
df_samples = pd.read_csv('sdt887_product_attribution_data_one_listing_per_leaf_node__20230111.csv')

In [47]:
df_samples = df_samples[df_samples.category_path.apply(lambda x: x in metadata_category)]
df_samples = df_samples.sample(10, random_state=42)

In [48]:
res = []
for i in df_samples[['product_id', 'category_id', 'category_path', 'title', 'product_description', 'main_image_url']].to_dict('records'):
    i['text'] = \
f"""Title: {i['title']}
Description: {i['product_description']}"""
    i['attribute_config'] = category2config[i['category_path']]
    res.append(i)

In [49]:
pd.DataFrame(res).to_json('example_offshore_input_listing_attr_extract.json', lines=True, orient='records')